# CNN Comparisons

## LeNet:
### Main idea:

At a high level, LeNet-5 consists of two parts:

* A convolutional encoder consisting of two convolutional layers.
* A dense block consisting of three fully connected layers. 


Data flow in LeNet. The input is a handwritten digit, the output a probability over 10 possible outcomes. 

__Convolutional Layer__: 

* Each convolutional layer uses a 5 x 5 kernel
* Sigmoid Activation function
* Average pooling.

In resume: 
    5 x 5 kernel -> Sigmoid activation function -> 2 x 2 average pooling (stride 2)

### Number of output channels: 
* 1st convolutional layer has 6 output channel
* 2nd                         16          

The convolutional block emits an output with shape given by (batch size, number of channel, height, width)

### Question: Why the number of output channel increases as the data moves from one convolutional layer to the other?

#### Answer: 
These layers map spatially arranged inputs to a number of two-dimensional feature maps, typically increasing the number of channels. 

### Dense Block: 

In order to pass output from the convolutional block to the dense block, we must flatten each example in the minibatch. In other words, we take this four-dimensional input and transform it into the two-dimensional input expected by fully connected layers: as a reminder, the two-dimensional representation that we desire uses the first dimension to index examples n the minibatch and the second to give the flat vector representation of each example. 

LeNet dense block has three fully connected layers, with 120, 84, and 10 outputs, respectively.

### Problems: 

* Small dataset
* Not enough computational power
* Low resolution images 28 x 28
* No key tricks for training neural networks (parameter initialization)
* No clever variants for SGD
* Non-squashing activation function
* Effective regularization techniques were still missing

Although LeNet-5 achieved good results on early small datasets, the performance and feasibility of training CNNs on larger, more realistic datasets had yet to be established.

Even though the inputs to convolutional networks consist of raw or lightly-processed (e.g., by centering) pixel values, practitioners would never feed _raw_ pixels into traditional models. Instead, typical computer vision pipelines consisted of manually engineering feature extraction pipelines, such as SIFT, and SURF. Rather than _learning the features_, the features were crafted. Most of the progress came from having more clever ideas for feature extraction on the one hand and deep insight into geometry on the other hand.The learning algorithm was often considered an afterthought.




## AlexNet:
### Main idea:

Although feature extraction tools ruled the roost prior to the introduction of AlexNet, another group of reseachers believed that features themselves ought to be learned instead of manually engineered. Moreover, they believed that to be reasonably complex, the features ought to be hierarchically composed with multiple jointly learned layers, each with learnable parameters. In the case of image, the lowest layers might come to detect edges, colors, and textures, in analogy to how the visual system in animal processes its input. 

AlexNet (2012) and its precursor LeNet (1995) share many architectural elements. This begs the question: why did it take so long? A key difference is that over the past two decades, the amount of data and computing power available had increased significantly. As such AlexNet was much larger: it was trained on much more data, and on much faster GPUs, compared to the CPUs available in 1995.

### LeNet-5 vs AlexNet:

* AlexNet is much deeper than the comparatively small LeNet5
* AlexNet consists of eight layers:
    * 5 convolutional layer
    * 2 fully connected hidden layers
    * 1 fully connected output layer
* AlexNet controls the model complexity of the fully connected layer by dropout, while LeNet only uses weight decay.
* AlexNet used the ReLU instead of the Sigmoid as its activation function. 

On the one hand, the computation of the ReLU activation function is simple. For example it does not have the exponentiation operation found in the sigmoid activation function. On the other hand, ReLU makes model training easier when using different parameter initialization methods.This is because, when the output of the sigmoid activation function is very close to 0 or 1, the gradient of these regions is almost 0, so that backpropagation cannot continue to update some of the model parameters. In contrast, the gradient of the ReLU activation function in the positive interval is always 1. Therefore, if the model parameters are not properly initialized, the sigmoid function may obtain a gradient of almost 0 in the positive interval, so that the model cannot be effectively trained.

__Convolutional Layer__:

* 1st layer: 11 x 11 kernel (since the images in ImageNet are eight times higher and wider than MNIST images.
* 2nd layer: 5 x 5
* 3rd layer: 3 x 3

After the 1st, 2nd, and 5th convolutional layers, the network adds max_pooling layers with:

* 3 x 3 window shape and stride of 2

After the last convolutional layer, there are two __huge__ fully connected layers with 4096 outputs. These layers require nearly 1GB model parameters. 

AlexNet’s structure bears a striking resemblance to LeNet, with a number of critical improvements, both for accuracy (dropout) and for ease of training (ReLU).


### Problems: 

Reviewing the architecture, we see that AlexNet has an Achilles heel when it comes to _efficiency_: 

* The last two layers require matrices of size 6400 x 4096 and 4096 x 4095, respectively. 


This corresponds to 164 MB of memory and 81 MLFLOPs of computation, both of which are a nontrivial outlay, especially on smaller devices, such as mobile phones. This is one of the reasons why AlexNet has been surpassed by much more effective architectures. 

Note that even though the number of parameters by far exceeds the amount of training data in our experiments (the last two layers have more than 40 million parameters, trained on a datasets of 60 thousand images), there is hardly any overfitting: training and validation loss are virtually identical throughout training. This is due to the improved regularization, such as Dropout, inherent in modern deep network designs.

## VGG:
### Main idea:

The basic building block of CNNs is a sequence of the following:

* a convolutional layer with padding to maintain the resolution
* a nonlinearity such as ReLU
* a pooling layer such as max-pooling to reduce the resolution

However, one of the problems with this approach is that the spatial resolution decreases quite rapidly. In particular, this imposes a hard limit of lg d convolutional layers on the network before all dimensions (d) are used up. For instance, in the case of ImageNet, it would be impossible to have more than 8 convolutional layers in this way.

The key of the paper introducing VGG was to use _multiple convolutions in between downsampling via max-pooling in the form of a block_. 

 They were primarily interested in whether deep or wide networks perform better. For instance, the successive application of two  convolutions touches the same pixels as a single  convolution does. At the same time, the latter uses approximately as many parameters () as three  convolutions do (). In a rather detailed analysis they showed that deep and narrow networks significantly outperform their shallow counterparts. This set deep learning on a quest for ever deeper networks with over 100 layers for typical applications. Stacking  convolutions has become a gold standard in later deep networks (a design decision only to be revisited recently by Liu et al. (2022)). Consequently, fast implementations for small convolutions have become a staple on GPUs (Lavin and Gray, 2016).

### VGG Architecture: 

A VGG block consists of a sequence of convolutions with:

* 3 x 3 kernels with padding 1 (keeping height and width)
* 2 x 2 max-pooling layer with stride of 2 (halving height and width after each block)

The original VGG network had 5 convolutional blocks, among which the first two have one convolutional layer each and the latter three contain two convolutional layers each. The first block has 64 output channels and each subsequent block doubles the number of output channels, until that number reaches 512. Since this network uses 8 convolutional layers and 3 fully connected layers, it is often called VGG-11.

### Problems:

LeNet, AlextNet and VGG all share a common design pattern: extract features exploiting _spatial structure_ via a sequence of convolutions and pooling layers and post-process the representations via fully connected layers.

This design poses two major challenges: 

* Fully connected layers at the end of the architecture consume tremendous numbers of parameters. This is a significant impediment to computation, in particular on mobile and embedded devices.
* It is equally impossible to add fully connected layers earlier in the network to increase the degree of nonlinearity: doing so would destroy the spatial structure and require potentially even memory.

## NiN:
### Main idea:

The _network in network_ blocks offer an alternative, capable of solving both problems (aforementioned) in one simple strategy. They were proposed based on a very simple insight:

1) Use 1 x 1 convolutions to add local nonlinearities across the channel activations.
2) Use global average pooling to integrate across all locations in the last representation layer. Note that global average pooling would not be effective, were it not for the added nonlinearities. 

As shown on the architectures above, the inputs and outputs of convolutional layers consist of four-dimensional tensors with axes corresponding to the example, channel, height, and width. In addition, the inputs and outputs of fully connected layers are typically two-dimensional tensors corresponding to the example and feature. 

The idea behind NiN is to apply a fully connected layer at each pixel location (for each height and width). The resulting 1 x 1 convolution can be thought as a fully connected layer acting independently on each pixel location.

## NiN Architecture: 

NiN uses the same initial convolution sizes as AlexNet. The kernel sizes are 11 x 11, 5 x 5, and 3 x 3, respectively, and the number of output channels match those of AlexNet. Each NiN block is followed by a max-pooling layer with a stride of 2 and a window shape of 3 x 3. 

## NiN vs VGG and AlexNet:

NiN avoids fully connected layers altogether. Instead, NiN uses a NiN block with a number of output channels equal to the number of label classes, followed by a global average pooling layer, yielding a vector of logits. This design significantly reduces the number of required model parameters, albeit at the expense of a potential increase in training time. 

## Problems:

* Increased training time

## GoogLeNet:
### Main idea:

In 2014, GoogLeNet won the ImageNet challenge, using a structure that combined the strenghts of NiN, repeated blocks, and a cocktail of convolution kernels. It was the first network to show a clear distinction among the data ingest, data processing, and prediction in a CNN. The data processing is given by the first 2-3 convolutions that operate on the image. They extract low-level features from the underlying images. This is followed by a body of convolutional blocks. Finally, the head maps the features obtained so far to the required classification, segmentation, detection, or tracking problem at hand. 


The key contribution in GoogLeNet was the desing of the network body. It solved the problem of selection convolution kernels in an ingenious way. While other works tried to identify which convolution, ranging from 1 x 1 to 11 x 11 would be best, it simply concatenated _multi-branch convolutions_